In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime, timedelta, timezone

# LAPD Arrests 

This notebook uploads, and cleans the data to look at the columns that pertain to the criminal activity, the arrests made, and the effects of the Safe Neighborhoods and Schools Act that Passed in 2014.

We have the following datasets from [data.lacity.org](data.lacity.org)
1. Arrests:
   - [Arrest Data from 2010 - 2019](https://data.lacity.org/Public-Safety/Arrest-Data-from-2010-to-2019/yru6-6re4/about_data) 
   - [Arrest Data from 2020 - Present](https://data.lacity.org/Public-Safety/Arrest-Data-from-2020-to-Present/amvf-fr72/about_data)
2. Crimes:
   - [Crime Data from 2010 to 2019](https://data.lacity.org/Public-Safety/Crime-Data-from-2010-to-2019/63jg-8b9z/about_data)
   - [Crime Data from 2020 to present](https://data.lacity.org/Public-Safety/Crime-Data-from-2020-to-Present/2nrs-mtv8/about_data)

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'analytic_env.yml',
 'arrests_all.csv',
 'Arrest_Data_from_2010_to_2019_20240611.csv',
 'Arrest_Data_from_2020_to_Present_20240611.csv',
 'crimes_all.csv',
 'crimes_and_arrests.csv',
 'Crime_Data_from_2010_to_2019_20240611.csv',
 'Crime_Data_from_2020_to_Present_20240611.csv',
 'LAPD 1.ipynb',
 'LAPD 2.ipynb',
 'mapping crime codes.ipynb',
 'mean cleaning up after a robot.png',
 'MO_CODES_Numerical_20191119 (1).pdf',
 'Tracking Prop 47.ipynb',
 'UCR-COMPSTAT062618 (1).pdf',
 'ucr_handbook_2013 (1).pdf',
 'Untitled.ipynb',
 'Untitled1.ipynb']

## Upload Raw Data:

In [3]:
a_2020 = pd.read_csv('Arrest_Data_from_2020_to_Present_20240611.csv')
a_2010 = pd.read_csv('Arrest_Data_from_2010_to_2019_20240611.csv')
c_2020 = pd.read_csv('Crime_Data_from_2020_to_Present_20240611.csv')
c_2010 = pd.read_csv('Crime_Data_from_2010_to_2019_20240611.csv')

In [70]:
c_2010.shape, c_2020.shape

((2122794, 28), (947775, 28))

In [71]:
a_2010.shape, a_2020.shape

((1320983, 25), (288101, 25))

## One column in conflict for crimes:

In [7]:
c_2010.columns == c_2020.columns

array([ True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [76]:
c_2010.columns[4] # 'AREA '

'AREA '

In [77]:
c_2020.columns[4]

'AREA'

In [8]:
a_2020.columns == a_2010.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

## Merge out of the gate.

Because the columns are identical, I'm tempted to merge the dataframes for arrests and crime, respectively to simplify things. Let's inspect the contents first.

In [45]:
a_2020.sample()

,Report ID,Report Type,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,...,Disposition Description,Address,Cross Street,LAT,LON,Location,Booking Date,Booking Time,Booking Location,Booking Location Code
136241,6060088,BOOKING,11/27/2020 12:00:00 AM,1720.0,4,Hollenbeck,423,33,M,H,...,MISDEMEANOR COMPLAINT FILED,MAIN,JOHNSTON ST,34.066,-118.2102,POINT (-118.2102 34.066),11/27/2020 12:00:00 AM,1957.0,METRO - JAIL DIVISION,4273.0


In [42]:
a_2010.sample()

,Report ID,Report Type,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,...,Disposition Description,Address,Cross Street,LAT,LON,Location,Booking Date,Booking Time,Booking Location,Booking Location Code
552116,2812579,BOOKING,07/17/2011,1930.0,10,West Valley,1045,33,M,W,...,OTHER (REQUIRES ADDITION ACTION),WHITE OAK AV,BURBANK BL,34.1723,-118.5185,POINT (-118.5185 34.1723),07/17/2011 12:00:00 AM,2139.0,VALLEY - JAIL DIV,4279.0


In [47]:
c_2020.sample()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
757805,230208692,04/08/2023 12:00:00 AM,04/08/2023 12:00:00 AM,730,2,Rampart,237,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,998.0,NaN,NaN,1300 W 2ND ST,NaN,34.0609,-118.2586


In [59]:
c_2010.sample()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
1715428,181816871,08/13/2018 12:00:00 AM,08/12/2018 12:00:00 AM,1900,18,Southeast,1821,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,700 W 102ND ST,NaN,33.9438,-118.2869


The contents of the data are comparable among the pairs. 

In [79]:
arrests_all_raw = a_2010.merge(a_2020, how = 'outer')

In [80]:
arrests_all_raw.to_csv('arrests_all_raw.csv', index = False)

In [87]:
c_2010.rename(columns = {'AREA ':'AREA'}, inplace = True)

In [88]:
c_2010.columns == c_2020.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [90]:
crime_reports_all_raw = c_2010.merge(right = c_2020, how = 'outer')

In [91]:
crime_reports_all_raw.to_csv('crime_reports_all_raw.csv', index=False)

# Dropping Columns

## arrests - dropping columns

In [92]:
%who

a_2010	 a_2020	 arrests_all_raw	 c_2010	 c_2020	 crime_reports_all_raw	 datetime	 np	 os	 
pd	 plt	 sns	 timedelta	 timezone	 


In [118]:
arrests_all_raw.columns = arrests_all_raw.columns.str.lower()
arrests_all_raw.columns = arrests_all_raw.columns.str.replace(' ', '_')

In [119]:
arrests=arrests_all_raw.copy()

In [126]:
arrests.columns

Index(['report_id', 'report_type', 'arrest_date', 'time', 'area_id',
       'area_name', 'reporting_district', 'age', 'sex_code', 'descent_code',
       'charge_group_code', 'charge_group_description', 'arrest_type_code',
       'charge', 'charge_description', 'disposition_description', 'address',
       'cross_street', 'lat', 'lon', 'location', 'booking_date',
       'booking_time', 'booking_location', 'booking_location_code'],
      dtype='object')

In [125]:
arrests[['report_id', 'report_type', 'arrest_date', 'time', 'area_id',
       'area_name', 'reporting_district', 'age', 'sex_code', 'descent_code',
       'charge_group_code', 'charge_group_description', 'arrest_type_code']].sample()

,report_id,report_type,arrest_date,time,area_id,area_name,reporting_district,age,sex_code,descent_code,charge_group_code,charge_group_description,arrest_type_code
562027,4388302,BOOKING,07/17/2015,2300.0,13,Newton,1323,38,M,B,16.0,Narcotic Drug Laws,F


In [129]:
arrests[['charge', 'charge_description', 'disposition_description', 'address',
       'cross_street', 'lat', 'lon', 'location', 'booking_date',
       'booking_time', 'booking_location', 'booking_location_code']].sample()

,charge,charge_description,disposition_description,address,cross_street,lat,lon,location,booking_date,booking_time,booking_location,booking_location_code
324358,459PC,BURGLARY,FELONY COMPLAINT FILED,2600 VAN BUREN PL,NaN,34.0328,-118.297,POINT (-118.297 34.0328),01/24/2013 12:00:00 AM,2000.0,SOUTHWEST,4203.0


In [130]:
# Columns to drop

arrest_columns_to_drop = ['area_id', 'reporting_district', 'charge_group_code', 'charge', 'address', 'cross_street', 'location', 'booking_date', 'booking_time', 'booking_location', 'booking_location_code']

In [133]:
arrests.drop(columns = arrest_columns_to_drop, inplace=True)

In [134]:
arrests.sample()

,report_id,report_type,arrest_date,time,area_name,age,sex_code,descent_code,charge_group_description,arrest_type_code,charge_description,disposition_description,lat,lon
967459,6133143,BOOKING,02/25/2021 12:00:00 AM,2045.0,Olympic,23,M,H,Driving Under Influence,M,DRUNK DRIVING ALCOHOL/DRUGS,MISDEMEANOR COMPLAINT FILED,0.0,0.0


## crimes - dropping columns

In [135]:
%who

a_2010	 a_2020	 arrest_columns_to_drop	 arrests	 arrests_all_raw	 c_2010	 c_2020	 crime_reports_all_raw	 datetime	 
np	 os	 pd	 plt	 sns	 timedelta	 timezone	 


In [312]:
crimes = crime_reports_all_raw.copy()

In [313]:
crimes.columns = crimes.columns.str.lower()
crimes.columns = crimes.columns.str.replace(' ', '_')

In [314]:
crimes.sample()

,dr_no,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1-2,crm_cd,crm_cd_desc,...,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
771182,131921353,10/15/2013 12:00:00 AM,10/04/2013 12:00:00 AM,1200,19,Mission,1998,2,649,DOCUMENT FORGERY / STOLEN FELONY,...,IC,Invest Cont,649.0,NaN,NaN,NaN,7600 WOODMAN AV,NaN,34.2111,-118.4309


In [315]:
# no information on this column at the website...
crimes['part_1-2'].value_counts(normalize=True)

part_1-2
1    0.564698
2    0.435302
Name: proportion, dtype: float64

In [316]:
crimes[['dr_no', 'date_rptd', 'date_occ', 'time_occ', 'area', 'area_name',
       'rpt_dist_no', 'part_1-2', 'crm_cd', 'crm_cd_desc', 'mocodes',
       'vict_age', 'vict_sex', 'vict_descent', 'premis_cd', 'premis_desc']].sample()

,dr_no,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1-2,crm_cd,crm_cd_desc,mocodes,vict_age,vict_sex,vict_descent,premis_cd,premis_desc
2563131,220405842,02/24/2022 12:00:00 AM,02/24/2022 12:00:00 AM,300,4,Hollenbeck,404,2,888,TRESPASSING,0913,40,M,H,501.0,SINGLE FAMILY DWELLING


In [148]:
crimes[['weapon_used_cd', 'weapon_desc', 'status', 'status_desc', 'crm_cd_1',
       'crm_cd_2', 'crm_cd_3', 'crm_cd_4', 'location', 'cross_street', 'lat',
       'lon']].sample()

,weapon_used_cd,weapon_desc,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
3047755,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AA,Adult Arrest,210.0,998.0,NaN,NaN,11100 BURBANK BL,NaN,34.1758,-118.3746


In [310]:
crimes_columns_to_drop = ['area', 'rpt_dist_no', 'part_1-2', 'crm_cd', 'mocodes', 'premis_cd', 'premis_desc', 'status', 'crm_cd_1', 'crm_cd_2', 'crm_cd_3', 'crm_cd_4','location', 'cross_street']

In [317]:
crimes.drop(columns = crimes_columns_to_drop, inplace=True)

In [318]:
crimes.sample()

,dr_no,date_rptd,date_occ,time_occ,area_name,crm_cd_desc,vict_age,vict_sex,vict_descent,weapon_used_cd,weapon_desc,status_desc,lat,lon
353539,111709598,04/18/2011 12:00:00 AM,04/17/2011 12:00:00 AM,1700,Devonshire,"BURGLARY FROM VEHICLE, ATTEMPTED",35,F,H,NaN,NaN,Invest Cont,34.2584,-118.4875


# rename columns

## renaming crime columns

In [319]:
# column mapper for crimes table:

mapper = {'dr_no' : 'record', 'date_rptd': 'report_date', 'date_occ': 'occured_date', 'time_occ': 'occured_time', 
          'area_name': 'area', 'crm_cd_desc': 'crime', 'vict_age': 'victim_age', 'vict_sex' : 'victim_sex', 'vict_descent': 
          'victim_descent', 'premis_desc' : 'premise', 'weapon_desc' : 'weapon', 'status_desc' : 'status', 'crm_cd_1': 'crime_code'}

In [320]:
crimes.rename(columns = mapper, inplace=True)

In [321]:
crimes.columns

Index(['record', 'report_date', 'occured_date', 'occured_time', 'area',
       'crime', 'victim_age', 'victim_sex', 'victim_descent', 'weapon_used_cd',
       'weapon', 'status', 'lat', 'lon'],
      dtype='object')

In [322]:
crimes.sample()

,record,report_date,occured_date,occured_time,area,crime,victim_age,victim_sex,victim_descent,weapon_used_cd,weapon,status,lat,lon
364193,111811383,05/10/2011 12:00:00 AM,05/09/2011 12:00:00 AM,1325,Southeast,ROBBERY,51,F,H,207.0,OTHER KNIFE,Invest Cont,33.9433,-118.2491


In [323]:
crimes.drop(columns = ['weapon_used_cd'], inplace=True)

In [324]:
arrests.rename(columns = {'sex_code':'sex', 'descent_code':'descent', 'charge_group_description':'charge_group', 'area_name':'area'}, inplace=True)

In [325]:
crimes.sample()

,record,report_date,occured_date,occured_time,area,crime,victim_age,victim_sex,victim_descent,weapon,status,lat,lon
1657908,180100631,03/09/2018 12:00:00 AM,03/09/2018 12:00:00 AM,210,Central,INTIMATE PARTNER - SIMPLE ASSAULT,24,F,X,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Adult Arrest,34.0409,-118.2574


# Column modifications for **arrests** :

## `categorical` modifications.
1. Update `descent` to it's corresponding ethnic name.
2. map `charge_type_mapping` to their full title.


In [187]:
arrests.sample(3)

,report_id,report_type,arrest_date,time,area,age,sex,descent,charge_group,arrest_type_code,charge_description,disposition_description,lat,lon
268280,3222038,BOOKING,07/07/2012,2105.0,Mission,21,M,H,Driving Under Influence,M,DRUNK DRIVING ALCOHOL/DRUGS,MISDEMEANOR COMPLAINT FILED,34.2215,-118.4612
895716,5783758,BOOKING,10/26/2019,1830.0,Wilshire,33,M,W,Larceny,M,SHOPLIFTING,MISDEMEANOR COMPLAINT FILED,34.0761,-118.3766
1134266,100114318,RFC,07/21/2010,2325.0,Central,51,M,B,NaN,M,NaN,MISDEMEANOR COMPLAINT FILED,34.0450,-118.2628


In [206]:
# 1. Type of Charge
charge_type_mapper = {
    'D': 'Dependent',
    'F': 'Felony',
    'I': 'Infraction',
    'M': 'Misdemeanor',
    'O': 'Other'
}


# 2. Descent Code (collapsing all Asian and South Asian nationalities to "Asian")
descent_code_mapper = {
    'A': 'Asian',              # Other Asian
    'B': 'Black',
    'C': 'Asian',              # Chinese
    'D': 'Asian',              # Cambodian
    'F': 'Asian',              # Filipino
    'G': 'Pacific Islander',   # Guamanian
    'H': 'Hispanic',
    'I': 'Native American',
    'J': 'Asian',              # Japanese
    'K': 'Asian',              # Korean
    'L': 'Asian',              # Laotian
    'O': 'Other',
    'P': 'Pacific Islander',
    'S': 'Pacific Islander',   # Samoan
    'U': 'Pacific Islander',   # Hawaiian
    'V': 'Asian',              # Vietnamese
    'W': 'White',
    'X': 'Unknown',
    'Z': 'Asian'               # Asian Indian
}


## maping changes

In [203]:
arrests.sample()

,report_id,report_type,arrest_date,time,area,age,sex,descent,charge_group,arrest_type_code,charge_description,disposition_description,lat,lon
573843,4436116,BOOKING,09/06/2015,625.0,Pacific,31,M,H,Driving Under Influence,Misdemeanor,DRUNK DRIVING ALCOHOL/DRUGS,MISDEMEANOR COMPLAINT FILED,33.9314,-118.416


In [199]:
arrests['arrest_type_code'] = arrests['arrest_type_code'].map(charge_type_mapper)

In [208]:
arrests['descent'] = arrests['descent'].map(descent_code_mapper)

## `datetimes`
4. Convert times from `float` to `datetime.time`
5. Convert `arrest_date` to `datetime.date`  

In [225]:
arrests['arrest_date'] = pd.to_datetime(arrests['arrest_date'], errors='coerce')

### the time column

In [229]:
arrests['time'] = arrests['time'].astype(str)
arrests['time'] = arrests['time'].str.replace('.0', '')

In [230]:
# zfill = zero fill the single hours
arrests['time'] = arrests['time'].str.zfill(4)

In [231]:
# insert a ":"
arrests['time'] = arrests['time'].apply(lambda x: x[:-2] + ':' + x[-2:])

In [232]:
arrests['time'] = arrests['time'].fillna('00:00')

In [236]:
arrests['time'] = arrests['time'].str.replace('24:00', '23:30')

In [245]:
# dropping null times

arrests = arrests[~(arrests['time'] == "0n:an")].copy()

In [247]:
arrests['time'] = pd.to_datetime(arrests['time'], format='%H:%M').dt.time # 24:00

In [258]:
arrests.shape[0]

1320787

In [257]:
null_date_arrests = arrests[arrests['arrest_date'].isna()]
arrests = arrests[~(arrests['arrest_date'].isna())]

In [259]:
null_date_arrests.to_csv('arrests_null_date.csv', index=False)

In [261]:
arrests = arrests.to_csv('arrests_dated_all.csv', index=False)

# Column clean up for crimes

In [326]:
crimes['report_date'] = pd.to_datetime(crimes['report_date'], errors='coerce')

C:\Users\agawr\AppData\Local\Temp\ipykernel_24672\1246815709.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  crimes['report_date'] = pd.to_datetime(crimes['report_date'], errors='coerce')


In [327]:
crimes['occured_date'] = pd.to_datetime(crimes['occured_date'], errors = 'coerce')

C:\Users\agawr\AppData\Local\Temp\ipykernel_24672\2937434516.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  crimes['occured_date'] = pd.to_datetime(crimes['occured_date'], errors = 'coerce')


In [328]:
crimes.sample()

,record,report_date,occured_date,occured_time,area,crime,victim_age,victim_sex,victim_descent,weapon,status,lat,lon
1105719,151225085,2015-11-04,2015-11-04,2135,77th Street,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",19,F,H,UNKNOWN WEAPON/OTHER WEAPON,Invest Cont,33.9747,-118.2809


### modifying `crimes['occured_time']`

- cast astype(str)
- zfill 4
- insert : in the middle.
- cast to datetime.time.

In [329]:
crimes['occured_time'] = crimes['occured_time'].astype(str)
crimes['occured_time'] = crimes['occured_time'].str.zfill(4)

In [330]:
# [-2:] are the last two
# [:-2] are the first two

crimes['occured_time'] = crimes['occured_time'].apply(lambda x: x[:-2] + ':' + x[-2:])

In [331]:
crimes['occured_time'] = pd.to_datetime(crimes['occured_time'], format="%H:%M").dt.time

In [332]:
crimes.sample(4)

,record,report_date,occured_date,occured_time,area,crime,victim_age,victim_sex,victim_descent,weapon,status,lat,lon
1137767,151518937,2015-09-27,2015-09-27,12:55:00,N Hollywood,SHOPLIFTING - PETTY THEFT ($950 & UNDER),0,M,O,NaN,Invest Cont,34.1524,-118.3671
396267,112117797,2011-10-13,2011-09-13,15:30:00,Topanga,BURGLARY FROM VEHICLE,23,F,W,NaN,Invest Cont,34.2010,-118.6169
1747613,180907275,2018-03-03,2018-03-01,13:00:00,Van Nuys,BURGLARY,36,F,W,OTHER CUTTING INSTRUMENT,Adult Arrest,34.1552,-118.4476
1766629,181104974,2018-01-24,2018-01-03,13:00:00,Northeast,"EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)",0,X,X,NaN,Adult Other,34.0912,-118.2111


## Mapping victim data

In [333]:
crimes['victim_descent'].value_counts()

victim_descent
H    1015972
W     703192
B     468393
O     278237
X     175154
A      71951
K      14508
F       6765
C       4950
I       1872
J       1779
V       1241
Z        619
P        608
U        392
G        153
D        104
L         86
S         85
-          5
Name: count, dtype: int64

In [334]:
%who

a_2010	 a_2020	 arrest_columns_to_drop	 arrests	 arrests_all_raw	 c_2010	 c_2020	 charge_type_mapper	 crime_columns_to_drop	 
crime_reports_all_raw	 crimes	 crimes_columns_to_drop	 datetime	 descent_code_mapper	 gender_mapper	 mapper	 np	 null_date_arrests	 
os	 pd	 plt	 sns	 timedelta	 timezone	 


In [335]:
crimes['victim_descent'].map(descent_code_mapper).value_counts(dropna=False)

victim_descent
Hispanic            1015972
White                703192
Black                468393
NaN                  324508
Other                278237
Unknown              175154
Asian                102003
Native American        1872
Pacific Islander       1238
Name: count, dtype: int64

In [336]:
crimes['victim_descent'] = crimes['victim_descent'].map(descent_code_mapper)

In [339]:
crimes[crimes['victim_descent'].isna()]['victim_descent'] = 'Unknown'

C:\Users\agawr\AppData\Local\Temp\ipykernel_24672\29473635.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crimes[crimes['victim_descent'].isna()]['victim_descent'] = 'Unknown'


In [347]:
crimes.dropna(inplace=True)

In [350]:
crimes.to_csv('crimes_all_cleaned.csv', index=False)